In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df_train = pd.read_csv("train.csv")
df_testA = pd.read_csv('test_A榜.csv')
df_testB = pd.read_csv('test_B榜.csv')
string_col = ['MON_12_CUST_CNT_PTY_ID',
'AI_STAR_SCO',
'WTHR_OPN_ONL_ICO',
'SHH_BCK',
'LGP_HLD_CARD_LVL',
'NB_CTC_HLD_IDV_AIO_CARD_SITU']

In [22]:
df_train.loc[:,'SHH_BCK']=df_train.loc[:,'SHH_BCK']-2
df_train.SHH_BCK

0        30.0
1        30.0
2         0.0
3        10.0
4         0.0
         ... 
39995    10.0
39996    10.0
39997    30.0
39998    10.0
39999    30.0
Name: SHH_BCK, Length: 40000, dtype: float64

缺失率超过25%，不要->bench is 75%

In [ ]:
def drop_too_few_variable(dataframe:pd.DataFrame, threshold=0.75,str_col=string_col, keep_stringcol=True):
    mask_drop = (dataframe.count()/40000)>threshold
    if keep_stringcol:
        _df_numerical = dataframe.loc[:,mask_drop]
    else:
        dataframe = dataframe.loc[:,mask_drop]


-2处理,SHHBCK也要-2，做one hot,分出label

In [1]:
def step1_data_processing(_df_numerical:pd.DataFrame):
    # slip 2 numerical and string columns
    _df_numerical = _df_numerical.iloc[:, 2:] - 2
    if 'LABEL' in _df_numerical.columns:
        df_lable = _df_numerical['LABEL']
    else:
        df_lable=None

    return _df_numerical, df_lable, _df_numerical
test_numerical, test_label, df_testB =  step1_data_processing(df_testB)


NameError: name 'pd' is not defined

clip by quantile up0.975, down0.02

In [9]:
# Quantile function
def clip_by_quantile(dataframe: pd.DataFrame, down=0.02, up=0.975):
    res_df = pd.DataFrame()
    for i in dataframe.columns:
        the_col = dataframe.loc[:, i]
        lo_lim = the_col.quantile([down, up])[down]
        up_lim = the_col.quantile([down, up])[up]
        res_df.loc[:, i] = the_col.clip(lower=lo_lim, upper=up_lim, inplace=False)
    return res_df
test_numerical = clip_by_quantile(test_numerical)

确定test和train的分布区别，留下形态差不多的
mask = df_test[:].dtypes != object
df_without_object = df_test.loc[:, mask]